<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/confirm_label_noize_for_cm_data/eras/confirm_label/examine_CM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#環境設定

In [1]:
%tensorflow_version 2.x

## import
# file dealing
import os
from os import path

import shutil
from google.colab import drive
import datetime
# data dealing
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
# process deasing
import gc
from time import sleep

# machine learning (back)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, initializers, callbacks

# machine learning
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical

import json
from collections import OrderedDict
import pprint
import re
import requests
import time
from IPython.display import clear_output

In [2]:
def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'cHdELzsau6ve8hNVL3FxPz65Jdyquzuj2kd021u8q1L'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': notification_message}
    requests.post(line_notify_api, headers = headers, data = data)

# 実験条件（外乱）を定める

In [3]:
LABEL_NOISE_RATE = 0.0
TEST_DATA_RATE = 0.25
EXPERIMENTS_NUMBER = 6

#学習条件を定める

##学習手法の仕様

In [4]:
seed = 20201218

learningDict = {
    "optimizer" : {
        "this.optimizer" : "sgd",
        "learning_rate" : "/notice! this is designed at before each learning",
        "momentum" : 0.9,
        "decay" : 1e-4,
        "nesterov" : True
    },
    "theWay" : {
        "batch_size" : 32,
        "epochs" : 128,
    },
    "compared_losses" : [
                         tf.keras.losses.CategoricalCrossentropy(),
                         tf.keras.losses.MeanSquaredError(), 
                         tf.keras.losses.MeanAbsoluteError(),
                        #  tf.keras.losses.SquaredHinge()               
    ]
}

# learningDict = {
#     "optimizer" : {
#         "this.optimizer" : "adam",
#         "learning_rate" : 0.001,
#         "epsilon" : 1e-8,
#         "beta_1" : 0.9,
#         "beta_2" : 0.999
#     },
#     "theWay" : {
#         "batch_size" : 32,
#         "epochs" : 128,
#     },
#     "compared_losses" : [
#                         tf.keras.losses.MeanAbsoluteError(),
#     ]
# }

def compile_optimizer():
  # 最適化処理 (adamのみ対応)
  if learningDict["optimizer"]["this.optimizer"] == "adam":
    optimizer = keras.optimizers.Adam(
        lr=learningDict["optimizer"]["learning_rate"],
        epsilon=learningDict["optimizer"]["epsilon"],
        beta_1=learningDict["optimizer"]["beta_1"],
        beta_2=learningDict["optimizer"]["beta_2"])
    print("adam is used as a optimizer")

  elif learningDict["optimizer"]["this.optimizer"] == "Nadam":
    optimizer = keras.optimizers.Nadam(
        lr=learningDict["optimizer"]["learning_rate"],
        beta_1=learningDict["optimizer"]["beta_1"],
        beta_2=learningDict["optimizer"]["beta_2"],
        epsilon=None, 
        schedule_decay=0.4)
    print("Nadam is used as a optimizer")

  elif learningDict["optimizer"]["this.optimizer"] == "sgd":
    optimizer = keras.optimizers.SGD(
        lr=learningDict["optimizer"]["learning_rate"],
        momentum=learningDict["optimizer"]["momentum"],
        decay=learningDict["optimizer"]["decay"],
        nesterov=learningDict["optimizer"]["nesterov"]) 
    print("sgd is used as a optimizer")
  else:
    print("error")
  
  return optimizer

## モデルの仕様（ニューラルネットワーク）
<ul>
  <li>入力層(フレームサイズ, フレームの高さ, フレームの横幅, RGB情報) </li>
  <li>出力層(予測値) </li>
  <li> 中間層 
    <ol>
      <li>conv0</li>
      <li>pool0</li>
      <li>conv1</li>
      <li>pool1</li>
      <li>dence0</li>
  </li>
</ui>

In [7]:
def make_model(video_format):
  # モデル作成
  model = models.Sequential()
  # 入力層
  model.add(
      layers.Reshape(
          (video_format.FRAME_SIZE,
          video_format.HEIGHT,
          video_format.WIDTH,
          video_format.COLORinfo),
          input_shape=(video_format.FRAME_SIZE * video_format.HEIGHT * video_format.WIDTH * video_format.COLORinfo,),
          name='Input_Layer' )
  )
  # 畳み込み0
  model.add(
      layers.Conv3D(
          filters=32,
          kernel_size=(3, 3, 3),
          strides=(1, 1, 1),
          padding='same',
          activation='relu',
          name='conv0'))
  # pool0
  model.add(
      layers.MaxPooling3D(pool_size=(2, 2, 2), name='pool0'))

  # 畳み込み1
  model.add(
      layers.Conv3D(
          filters=32,
          kernel_size=(3, 3, 3),
          strides=(1, 1, 1),
          padding='same',
          activation='relu',
          name='conv1'))
  # pool1
  model.add(
      layers.MaxPooling3D(pool_size=(2, 2, 2), name='pool1'))

  ## 全結合0
  model.add(
      layers.Flatten(name='pipe'),
  )
  model.add(
      layers.Dense(1024,
        activation='relu',
        name='dence0' ),
  )
  # 出力層
  model.add(
      layers.Dense(4, activation='softmax', name='WATERSUPPLY')
  )
  return model


##データの仕様

In [8]:
## フォーマットの設定
class video_format:
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"

# 学習データの用意

In [9]:
## gdrive 接続
if not path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Already confirm")

## colab テンポラリディレクトリの作成
desk = '/content/desk'
if not os.path.exists(desk):
  os.mkdir(desk)
os.chdir(desk)
print("Created at /content/desk")

Mounted at /content/drive
Created at /content/desk


In [10]:
learning_data_path = "/content/drive/MyDrive/colab/CM_experiment/cleaned_CM"

In [11]:
if path.isdir(learning_data_path):
  print("actually exist the", learning_data_path)
  for each_data in os.listdir(learning_data_path):
    if re.match(r"Data.*\.npz", each_data):
      print("________|------------ reading [", each_data, "] as learning data.")
      learning_data_np = np.load(path.join(learning_data_path, each_data))
    elif re.match(r"Label.*\.npz", each_data):
      print("________|------------ reading [", each_data, "] as label data.")
      label_data_np = np.load(path.join(learning_data_path, each_data))
    else:
      print("Not reading such data", each_data)
else:
  print("no such path")


actually exist the /content/drive/MyDrive/colab/CM_experiment/cleaned_CM
Not reading such data summary_CM_data0131.png
________|------------ reading [ Data0131.npz ] as learning data.
________|------------ reading [ Label0131.npz ] as label data.


In [12]:
learning_data = []
label_data = []
for i in learning_data_np.files:
  learning_data.append(learning_data_np[i])
for i in label_data_np.files:
  label_data.append(label_data_np[i])

learning_data = np.array(learning_data)
label_data = np.array(label_data)

## 訓練データとテストデータとで分割

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(learning_data, label_data, random_state=20200120, train_size=(1-TEST_DATA_RATE))

## テストデータの教師ラベルに意図的なノイズを加える(実験のために)

In [14]:
import random
def rand_ints_nodup(a, b, k):
  ns = []
  while len(ns) < k:
    n = random.randint(a, b)
    if not n in ns:
      ns.append(n)
  return ns
def changed_number(original_num, set_min, set_max):
  while True:
    tmpRndVal = random.randint(set_min, set_max)
    if original_num != tmpRndVal:
      return tmpRndVal

In [15]:
changed_label_number_for_experiment = int(len(Y_train) * LABEL_NOISE_RATE)
change_points = rand_ints_nodup(0, len(Y_train)-1, changed_label_number_for_experiment )
print("change map:", sorted(change_points))
print("the size:", len(change_points))

set_min = np.min(Y_train)
set_max = np.max(Y_train)
sum = 0
if LABEL_NOISE_RATE != 0:
  for i in range(len(Y_train)):
    if i in change_points:
      print("No.", i, ", original number is", Y_train[i], end=" -> ")
      Y_train[i] = changed_number(Y_train[i], set_min, set_max)
      print("changed number is", Y_train[i])
      sum += 1
    else:
      pass
else:
  print("No label noizes")
  
if sum == len(change_points):
  print("changed correctlly.")

change map: []
the size: 0
No label noizes
changed correctlly.


##教師ラベルをone-hotに変換

In [16]:
Y_train = tf.keras.utils.to_categorical(Y_train, 4)
Y_test = tf.keras.utils.to_categorical(Y_test, 4)

# 学習開始

In [17]:
def fix_loss_text(original):
  result = re.sub(r"<tensorflow\.python\.keras\.losses\.", "", str(original))
  result = re.sub(r"\sobject.+", "", result)
  return result

In [18]:
used_losses_set = []
for each_loss in learningDict["compared_losses"]:
  used_losses_set.append(fix_loss_text(each_loss))

In [ ]:
%%time
obj_video_format = video_format()
all_histories = [] # append EXPERIMENTS_NUMBER * used losses size

start_time = time.time()
for j in range(EXPERIMENTS_NUMBER):
  seed_in_roop = seed + j
  np.random.seed(seed_in_roop)
  tf.random.set_seed(seed_in_roop)
  print("runnng...")

  roop_histories = [] # append used losses size
  for i, each_loss in enumerate(learningDict["compared_losses"]):

    # 通知
    all_loop_counter = str(j * len(learningDict["compared_losses"]) + (i + 1))
    all_loop_number = str(EXPERIMENTS_NUMBER * len(learningDict["compared_losses"]))
    
    start_massage = "Try the " + all_loop_counter + "/" + all_loop_number + " loop\n"
    start_massage += "Current used loss function is [" + fix_loss_text(each_loss) + "] = " + str(i+1) +  "/" + str(len(learningDict["compared_losses"]))

    print(start_massage)
    send_line_notify(start_massage)

    try:
      # モデル構築
      model = make_model(obj_video_format)

      if fix_loss_text(each_loss) == "CategoricalCrossentropy":
        learningDict["optimizer"]["learning_rate"] = (1e-3)*2
        optimizer = compile_optimizer()
        print(learningDict["optimizer"]["learning_rate"])
      else:
        learningDict["optimizer"]["learning_rate"] = (1e-2)
        optimizer = compile_optimizer()
        print(learningDict["optimizer"]["learning_rate"])

      model.compile(
            optimizer=optimizer,
            loss=each_loss,
            metrics=['acc'])
      # 実行
      
      history = model.fit(
            X_train, Y_train,
            validation_data=(X_test, Y_test),
            batch_size=learningDict["theWay"]["batch_size"],
            epochs=learningDict["theWay"]["epochs"]
            # verbose=0
            )
    except KeyboardInterrupt: 
      print("\n\nProcessing the KeyboardInterrupt")

    else:
      roop_histories.append(history)
      accumulation_time = str(datetime.timedelta(seconds=(time.time() - start_time)))
      finish_massage = "Complete.\n"
      finish_massage += "The accumulation time is " + str(accumulation_time) + "\n"
      finish_massage += "Last val_acc is " + str(history.history["val_acc"][learningDict["theWay"]["epochs"]-1])
      print(finish_massage)
      send_line_notify(finish_massage)
    finally:
      del model
      keras.backend.clear_session()
      gc.collect()
      sleep(10)
      clear_output()
      print(" and the model is erased.")

  #/for i
  all_histories.append(roop_histories)

#/for j


 and the model is erased.
Try the 2/18 loop
Current used loss function is [MeanSquaredError] = 2/3
sgd is used as a optimizer
0.01
Epoch 1/128
6/6 [==============================] - 1s 192ms/step - loss: 0.1752 - acc: 0.4323 - val_loss: 0.1751 - val_acc: 0.4062
Epoch 2/128
6/6 [==============================] - 1s 172ms/step - loss: 0.1705 - acc: 0.4178 - val_loss: 0.1821 - val_acc: 0.4062
Epoch 3/128
6/6 [==============================] - 1s 168ms/step - loss: 0.1665 - acc: 0.4157 - val_loss: 0.1759 - val_acc: 0.4062
Epoch 4/128
6/6 [==============================] - 1s 167ms/step - loss: 0.1628 - acc: 0.4645 - val_loss: 0.1685 - val_acc: 0.4219
Epoch 5/128
6/6 [==============================] - 1s 170ms/step - loss: 0.1576 - acc: 0.5759 - val_loss: 0.1668 - val_acc: 0.4219
Epoch 6/128
6/6 [==============================] - 1s 169ms/step - loss: 0.1452 - acc: 0.5713 - val_loss: 0.1718 - val_acc: 0.3750
Epoch 7/128
6/6 [==============================] - 1s 168ms/step - loss: 0.1479 - a

## @学習可視化（history を一つだけ見る）

In [ ]:
IO = False

In [ ]:
now_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
today = now_time.strftime('%m%d')

print(today)
def plot_learning(history, experiment_name="No name"):
  HEIGHT = 1
  WIDTH = 2
  rate = 5.0
  WpH_rate = 1.5
  fig = plt.figure(figsize=(WIDTH*rate*WpH_rate, HEIGHT*rate))
  plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=None)

  LOSS = fig.add_subplot(HEIGHT, WIDTH, 1) # loss, val_loss
  ACC = fig.add_subplot(HEIGHT, WIDTH, 2) # acc, val_acc

  # 1,1 loss
  loss = history.history["loss"]
  val_loss = history.history["val_loss"]

  loss_props = {
        'title' : 'Loss values plot',
        'xlabel' : 'epoch',
        'ylabel' : 'value'
    }
  LOSS.set(**loss_props)
  LOSS.plot(loss, label='loss', color='blue')
  LOSS.plot(val_loss, label='val_loss', color='orange')
  LOSS.legend(loc='best')

  # 1,2 acc
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  acc_props = {
        'title' : 'Accuracy values plot',
        'xlabel' : 'epoch',
        'ylabel' : 'value'
    }

  ACC.set(**acc_props)
  ACC.plot(acc, label='acc', color='blue')
  ACC.plot(val_acc, label='val_acc', color='orange')
  ACC.legend(loc='best')

  #save
  image_path = path.join(desk, experiment_name+today)
  fig.savefig(image_path, bbox_inches='tight')

In [ ]:
if IO:
  plot_learning(histories[0], "how_adam_used_for_KTH_in_certain_model") 

## @学習可視化データ保存

In [ ]:
if IO:
  shelf = '/content/drive/MyDrive/colab'
  book = 'tuning_results'
  shelf_book = os.path.join(shelf, book)
  shelf_book_page = path.join(shelf_book, today)
  print(shelf_book_page)
  # 保存
  if not os.path.exists(shelf_book_page):
    os.makedirs(shelf_book_page)

  print("writing all to a strage now.")
  for each_file in os.listdir(desk):
    if re.match(r"\..*", each_file,):
      pass
    elif re.match(r".*\.png", each_file,):
      print("->", each_file)
      shutil.copy2(each_file, shelf_book_page)
    else:
      pass

# 学習結果保存

In [ ]:
now_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
today = now_time.strftime('%m%d')

for key_loss, each_loss in enumerate(used_losses_set):
  print(each_loss)
  for key_experient_number, each_experiment in enumerate(all_histories):
    print("   ", end="")
    print("experient_number :", key_experient_number)
    each_history = all_histories[key_experient_number][key_loss]
    hist_df = pd.DataFrame(each_history.history)

    name = "KTH" + "."
    name = name + str(LABEL_NOISE_RATE) + "."
    name = name + each_loss + "."
    name = name + str(key_experient_number) + "."
    name = name + today
    name = name + ".csv"

    hist_df.to_csv(name)

In [ ]:
print("all in the desk:")
files_in_the_desk = sorted(os.listdir(desk))
for each in files_in_the_desk:
  print("_____", end="")
  print(each)

In [ ]:
directory_name = "noize=" + str(LABEL_NOISE_RATE)
shelf_path = "/content/drive/MyDrive/colab/CM_experiment/histories"
shelf_path = path.join(shelf_path, directory_name)

if not path.exists(shelf_path):
  os.makedirs(shelf_path)
else: pass

for each_file in files_in_the_desk:
  shutil.copy2(each_file, shelf_path)


massage = "Histories are written\n"
massage += "Total time is " + str(datetime.timedelta(seconds=(time.time() - start_time)))

send_line_notify(massage)